In [ ]:
%matplotlib inline
from google.colab import files
import os

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd

# Sklearn
from sklearn.model_selection import train_test_split # Helps with organizing data for training
from sklearn.metrics import confusion_matrix # Helps present results as a confusion-matrix
import pickle
import onnx

print(tf.__version__)

In [ ]:
with open('X_test.pkl', 'rb') as file:
    X_test = pickle.load(file)

with open('y_test.pkl', 'rb') as file:
    y_test = pickle.load(file)

with open('X_train.pkl', 'rb') as file:
    X_train = pickle.load(file)

with open('y_train.pkl', 'rb') as file:
    y_train = pickle.load(file)


In [ ]:
model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(96, 96, 1))) 
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [ ]:
# Configures the model for training
model.compile(optimizer='adam', # Optimization routine, which tells the computer how to adjust the parameter values to minimize the loss function.
              loss='sparse_categorical_crossentropy', # Loss function, which tells us how bad our predictions are.
              metrics=['accuracy']) # List of metrics to be evaluated by the model during training and testing.

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 92, 92, 32)        832       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 46, 46, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 46, 46, 32)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 22, 22, 64)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 22, 22, 64)       

In [ ]:
# Trains the model for a given number of epochs (iterations on a dataset) and validates it.
history=model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1, validation_data=(X_test, y_test))

Epoch 1/5
132/132 [==============================] - 90s 675ms/step - loss: 3.0592 - accuracy: 0.6982 - val_loss: 0.0253 - val_accuracy: 0.9948
Epoch 2/5
132/132 [==============================] - 85s 642ms/step - loss: 0.0127 - accuracy: 0.9963 - val_loss: 0.0043 - val_accuracy: 0.9988
Epoch 3/5
132/132 [==============================] - 84s 635ms/step - loss: 0.0039 - accuracy: 0.9986 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 4/5
132/132 [==============================] - 84s 634ms/step - loss: 0.0056 - accuracy: 0.9986 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 5/5
132/132 [==============================] - 84s 634ms/step - loss: 0.0035 - accuracy: 0.9988 - val_loss: 4.5812e-04 - val_accuracy: 1.0000


In [ ]:
model.save('handrecognition_model.h5')

In [ ]:
model = tf.keras.models.load_model("/content/handrecognition_model.h5")
tf.saved_model.save(model, "tmp_model")

In [ ]:
!python -m tf2onnx.convert --saved-model tmp_model --output "handrecognition_model.onnx"

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-11-14 11:47:49,871 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2022-11-14 11:47:50,313 - INFO - Signatures found in model: [serving_default].
2022-11-14 11:47:50,313 - WARNING - '--signature_def' not specified, using first signature: serving_default
2022-11-14 11:47:50,313 - INFO - Output names: ['dense_1']
2022-11-14 11:47:50,504 - INFO - Using tensorflow=2.9.2, onnx=1.12.0, tf2onnx=1.13.0/2c1db5
2022-11-14 11:47:50,504 - INFO - Using opset <onnx, 13>
2022-11-14 11:47:50,689 - INFO - Computed 0 values for constant folding
2022-11-14 11:47:50,808 - INFO - Optimizing ONNX model
2022-11-14 11:47:50,864 - INFO - After optimization: Cast -1 (1->0), Const +1 (11->12), Identity -2 (2->0), Reshape +1 (1->2), Transpose -11 (12->1

In [ ]:
!zip -r /content/tmp_model.zip /content/tmp_model

  adding: content/tmp_model/ (stored 0%)
  adding: content/tmp_model/variables/ (stored 0%)
  adding: content/tmp_model/variables/variables.data-00000-of-00001 (deflated 6%)
  adding: content/tmp_model/variables/variables.index (deflated 66%)
  adding: content/tmp_model/assets/ (stored 0%)
  adding: content/tmp_model/saved_model.pb (deflated 89%)


In [ ]:
from google.colab import files
files.download("/content/handrecognition_model.h5")
files.download("/content/handrecognition_model.onnx")
files.download("/content/tmp_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>